# 6-1 순환 신경망
## 1. 순환 신경망(Recurrent Neural Network, RNN)
- 은닉층에서 활성화 함수를 지난 값은 오직 출력층 방향으로 향했는데 이와 같은 신경망들을 피드 포워드 신경망이라고 한다.
- RNN은 은닉층의 노드에서 활성화 함수를 통해 나온 결과값을 출력층 방향으로도 보내면서, 다시 은닉층 노드의 다음 계산의 입력으로 보내는 특징을 갖고 있다.
- RNN에서 은닉층에서 활성화 함수를 통해 결과를 내보내는 역할을 하는 노드를 셀(cell)이라고 한다.
- 셀은 이전의 값을 기억하려고 하는 일종의 메모리 역할을 수행하므로 메모리 셀 또는 RNN 셀이라고 표현한다.
- 은닉층의 메모리 셀은 각각의 시점(tiem step)에서 바로 이전 시점에서의 은닉층의 메모리 셀에서 나온 값을 자신의 입력으로 사용하는 재귀적 활동을 한다.
- 메모리 셀이 출력층 방향 또는 다음 시점 t+1의 자신에게 보내는 값을 은닉 상태(hidden state)라고 한다.
- RNN에서는 뉴런이라는 단위보다는 입력층과 출력층에서는 각각 입력 벡터와 출력 벡터, 은닉층에서는 은닉 상태라는 표현을 주로 사용한다.
- 은닉층

  $h_t = tanh(W_xx_t + W_hh_{t-1} + b)$
- 출력층

  $y_t = f(W_yh_t + b)$
  단, f는 비선형 활성화 함수 중 하나.
- $h_t$를 계산하기 위한 활성화 함수로는 주로 하이퍼볼릭탄젠트 함수가 사용되지만, ReLU로 바꿔 사용하는 시도도 있다.
- 각각의 가중치 $W_x, W_h, W_y$의 값은 모든 시점에서 값을 동일하게 공유한다.


In [3]:
# 2. 파이썬으로 RNN 구현하기
import numpy as np
timesteps = 10 # 시점의 수. NLP에서는 보통 문장의 길이가 된다.
input_size = 4 # 입력의 차원. 보통 단어 벡터의 차원이 된다.
hidden_size = 8 # 은닉 상태의 크기. 메모리 셀의 용량

inputs = np.random.random((timesteps, input_size)) # 입력에 해당되는 2D 텐서
hidden_state_t = np.zeros((hidden_size, )) # 초기 은닉 상태는 0(벡터)로 초기화

print(hidden_state_t)

[0. 0. 0. 0. 0. 0. 0. 0.]


In [6]:
Wx = np.random.random((hidden_size, input_size)) # (8, 4)크기의 2D 텐서 생성. 입력에 대한 가중치
Wh = np.random.random((hidden_size, hidden_size)) # (8, 8)크기의 2D 텐서 생성. 은닉 상태에 대한 가중치
b = np.random.random((hidden_size,)) # (8, )크기의 1D 텐서 생성. 편향

print(np.shape(Wx))
print(np.shape(Wh))
print(np.shape(b))

(8, 4)
(8, 8)
(8,)


In [7]:
total_hidden_states = []

# 메모리 셀 동작
for input_t in inputs:
  output_t = np.tanh(np.dot(Wx, input_t) + np.dot(Wh, hidden_state_t) + b)
  total_hidden_states.append(list(output_t)) # 각시점의 은닉 상태의 값을 계속해서 축적
  print(np.shape(total_hidden_states)) # 각 시점 t별 메모리 셀의 출력의 크기
  hidden_state_t = output_t

total_hidden_states = np.stack(total_hidden_states, axis=0)

print(total_hidden_states)

(1, 8)
(2, 8)
(3, 8)
(4, 8)
(5, 8)
(6, 8)
(7, 8)
(8, 8)
(9, 8)
(10, 8)
[[0.97473801 0.80855673 0.85124951 0.94741352 0.80719304 0.89963183
  0.7915908  0.77092437]
 [0.99964888 0.99973791 0.99991663 0.99963418 0.99964201 0.9999096
  0.99976492 0.99962991]
 [0.99987899 0.99996783 0.99997496 0.99986461 0.99998069 0.99999
  0.99994807 0.99990758]
 [0.99997648 0.99998316 0.99999053 0.99997142 0.99999208 0.99999701
  0.99998259 0.9999766 ]
 [0.9999379  0.99996891 0.99997728 0.99995194 0.99997336 0.99998776
  0.99997281 0.99991717]
 [0.99995921 0.99997857 0.99998958 0.99994593 0.99996943 0.99999075
  0.9999716  0.99997652]
 [0.99993138 0.99994582 0.99997801 0.99993655 0.99994972 0.99998609
  0.99996169 0.99992041]
 [0.99994843 0.99997187 0.99998265 0.99994446 0.9999895  0.99999547
  0.99997094 0.99994105]
 [0.99998254 0.99998412 0.99998937 0.99998751 0.99997581 0.99999079
  0.99998967 0.99997386]
 [0.99990147 0.99995877 0.9999822  0.99986255 0.99995427 0.9999867
  0.99994117 0.99994716]]


In [8]:
# 파이토치의 nn.RNN()
import torch
import torch.nn as nn

input_size = 5 # 입력의 크기
hidden_size = 8 # 은닉 상태의 크기

# 입력 텐서 정의
# (batch_size, time_steps, input_size)
inputs = torch.Tensor(1, 10, 5)
# batch_first=True -> 입력 텐서의 첫번째 차원이 배치 크기임을 알려줌
cell = nn.RNN(input_size, hidden_size, batch_first=True)
# outputs: 모든 시점의 은닉 상태들
# _status: 마지막 시점의 은닉 상태
outputs, _status = cell(inputs)

print(outputs.shape) # 모든 time-step의 hidden_state
print(_status.shape) # 최종 time-step의 hidden_state

torch.Size([1, 10, 8])
torch.Size([1, 1, 8])


In [16]:
# 4. 깊은 순환 신경망
# 깊은 순환 신경망을 파이토치로 구현할때는 nn.RNN()의 인자인 num_layers에 값을 전달하여 층을 쌓음.
# 입력 데이터
inputs = torch.Tensor(1, 10, 5)
cell = nn.RNN(input_size = 5, hidden_size = 8, num_layers = 2, batch_first=True)
outputs, _status = cell(inputs)
print(outputs.shape)
print(_status.shape) # (층의 개수, 배치 크기, 은닉 상태의 크기)

torch.Size([1, 10, 8])
torch.Size([2, 1, 8])


In [18]:
# 5. 양방향 순환 신경망
# 시점 t에서의 출력값을 예측할 때 이전 시점의 데이터뿐만 아니라, 이후 데이터로도 예측할 수 있다는 아이디어에 기반
# 양방향 RNN은 하나의 출력값을 예측하기위해 기본적으로 두 개의 메모리 셀을 사용
# 첫번째 메모리 셀은 앞에서 배운 것처럼 앞 시점의 은닉 상태를 전달받아 현재의 은닉 상태 계산
# 두번째 메모리 셀은 뒤 시점의 은닉 상태를 전달 받아 현재의 은닉 상태를 계산
# 이 두 개의 값 모두가 출력층에서 출력값을 예측하기 위해 사용된다.
# nn.RNN()의 인자인 bidirectional에 값을 True로 전달하면 된다.

# 학습 데이터
inputs = torch.Tensor(1, 10, 5)
cell = nn.RNN(input_size=5, hidden_size=8, num_layers=2, batch_first=True, bidirectional=True)
outputs, _status = cell(inputs)
print(outputs.shape) # (배치 크기, 시퀀스 길이, 은닉 상태의 크기 x 2)
print(_status.shape) # (층의 개수 * 2, 배치 크기, 은닉 상태의 크기)


torch.Size([1, 10, 16])
torch.Size([4, 1, 8])


# 6-2 LSTM과 GRU

## 1. 바닐라 RNN의 한계
- 바닐라 RNN은 비교적 짧은 시퀀스에 대해서만 효과를 보이는 단점이 있다.
- 바닐라 RNN의 시점이 길어질 수록 앞의 정보가 뒤로 충분히 전달되지 못하는 현상이 발생한다.
- 이를 **장기 의존성 문제**라고 한다.

## 2. LSTM(Long Short-Term Memory)
- LSTM은 은닉층의 메모리 셀에 입력 게이트, 망각 게이트, 출력 게이트를 추가하여 불필요한 기억을 지우고, 기억해야할 것을 정한다.
- LSTM은 은닉 상태를 계산하는 식이 전통적인 RNN보다 조금 더 복잡해졌으며 셀 상태라는 값을 추가했다.
- LSTM은 RNN과 비교하여 긴 시퀀스의 입력을 처리하는데 탁월한 성능을 보인다.
- 셀 상태 또한 은닉 상태처럼 이전 시점의 셀 상태가 다음 시점의 셀 상태를 구하기 위한 입력으로서 사용된다.
- 은닉 상태값과 셀 상태값을 구하기 위해서 새로 추가 된 3개의 게이트를 사용한다. 각 게이트는 삭제 게이트, 입력 게이트, 출력 게이트라고 부르며 이 3개의 게이트에는 공통적으로 시그모이드 함수가 존재한다.
- 시그모이드 함수를 지나면 0과 1사이의 값이 나오게 되는 이 값들을 가지고 게이트를 조절한다.
- 1) 입력 게이트
  - $i_t = σ(W_{xi}x_t + W_{hi}h_{t-1} + b_i)$
  - $g_t = tanh(W_{xg}x_t + W_{hg}h_{t-1} + b_g)$
  - 입력 게이트는 현재 정보를 기억하기 위한 게이트이다.
  - 현재 시점 t의 x값과 입력 게이트로 이어지는 가중치 $W_{xi}$를 곱한 값과 이전 시점의 t-1의 은닉 상태가 입력 게이트로 이어지는 가중치 $W_{hi}$를 곱한 값을 더하여 시그모이드 함수를 지난다. 이를 $i_t$라고 한다.
  - 현재 시점 t의 x값과 입력 게이트로 이어지는 가중치 $W_{xi}$를 곱한 값과 이전 시점 t-1의 은닉 상태가 입력 게이트로 이어지는 가중치 $W_{hg}$를 곱한 값을 더하여 하이퍼볼릭탄젠트 함수를 지난다. 이를 $g_t$라고 한다.
  - 시그모이드 함수를 지나 0과 1 사이의 값과 하이퍼볼릭탄젠트 함수를 지나 -1과 1 사이의 값 두개가 나오게 된다. 이 두 개의 값을 가지고 이번에 선택된 기억할 정보의 양을 정한다.
- 2) 삭제 게이트
  - $f_t = σ(W_{xf}x_t + W_{hf}h_{t-1} + b_f)$
  - 삭제 게이트는 기억을 삭제하기 위한 게이트이다
  - 현재시점 t의 x값과 이전 시점 t-1의 은닉 상태가 시그모이드 함수를 지나 0과 1 사이의 값이 나오게 되는데, 이 값이 삭제 과정을 거친 정보의 양이다.
  - 0에 가까울수록 정보가 많이 삭제된 것이고 1에 가까울수록 정보를 온전히 기억한 것이다.
- 3) 셀 상태(장기 상태)
  - $C_t = f_t∘C_{t-1} + i_t∘g_t$
  - 셀 상태 $C_t$를 LSTM에서는 장기 상태라고 부르기도 한다.
  - 입력 게이트에서 구한 $i_t, g_t$ 이 두 개의 값에 대해서 원소별 곲을 진행한다.
  - 입력 게이트에서 선택된 기억을 삭제 게이트의 결과값과 더한다. 이 값을 현재 시점 t의 셀 상태라고 하며, 이 값은 다음 t+1 시점의 LSTM 셀로 넘겨진다.
- 4) 출력 게이트와 은닉 상태(단기 상태)
  - $o_t = σ(W_{xo}x_t + W_{ho}h_{t-1} + b_o)$
  - $h_t = o_t∘tanh(c_t)$
  - 출력 게이트는 현재 시점 t의 x값과 이전 시점 t-1의 은닉 상태가 시그모이드 함수를 지난 값이다.
  - 해당 값은 현재 시점 t의 은닉 상태를 결정하는 일에 쓰인다
  - 은닉 상태를 단기 상태라고도 한다. 은닉 상태는 장기 상태의 값이 하이퍼볼릭탄젠트 함수를 지나 -1과 1사이의 값이다.
  - 해당 값은 출력 게이트의 값과 연산되면서, 값이 걸러지는 효과가 발생한다.
  - 단기 상태의 값은 또한 출력층으로도 향한다.

## 3. 파이토치의 nn.LSTM()
- 기존 RNN 사용 방법과 유사하게 사용하면 된다.
- `nn.LSTM(input_dim, hidden_size, batch_first=True)`
- GRU는 LSTM의 장기 의존성 문제에 대한 해결책을 유지하면서, 은닉 상태를 업데이트하는 계산을 줄였다. 즉, GRU는 성능은 LSTM과 유사하면서 복잡했던 LSTM의 구조를 간단화 시켰다

## 4. GRU(Gated Recurrent Unit)
- GRU에서는 업데이트 게이트와 리셋 게이트 두 가지 게이트만이 존재한다.
- $r_t = σ(W_{xr}x_t + W_{hr}h_{t-1} + b_r)$
- $z_t = σ(W_{xz}x_t + W_{hz}h_{t-1} + b_z)$
- $g_t = tanh(W_{hg}(r_t∘h_{t-1}) + W_{xg}x_t + b_g)$
- $h_t = (1 - z_t)∘g_t + z_t∘h_{t-1}$
- GRU와 LSTM 중 어떤 것이 모델의 성능면에서 더 낫다라고 단정지어 말할 수 없으며, 기존에 LSTM을 사용하면서 최적의 하이퍼파라미터를 찾아낸 상황이라면 굳이 GRU로 바꿔서 사용할 필요는 없다.
- 경험적으로 데이터 양이 적을 때는 매개 변수의 양이 적은 GRU가 조금 더 낫고, 데이터 양이 많으면 LSTM이 더 낫다고 한다.

## 5. 파이토치의 nn.GRU()
- `nn.GRU(input_dim, hidden_size, batch_first=True)`

# 6-3 문자 단위 RNN(Char RNN): 실습
- RNN의 입출력의 단위가 단어 레벨이 아니라 문제 레벨로 하여 RNN을 구현한다면, 이를 문자 단위 RNN이라고 한다.

In [55]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np

# 1) 훈련 데이터 전처리 하기
input_str = 'apple'
label_str = 'pple!'
char_vocab = sorted(list(set(input_str + label_str)))
vocab_size = len(char_vocab)
print('문자 집합의 크기 : {}'.format(vocab_size))

문자 집합의 크기 : 5


In [56]:
input_size = vocab_size # 입력의 크기는 문자 집합의 크기(입력은 원-핫 벡터 사용)
hidden_size = 5
output_size = 5
learning_rate = 0.1

char_to_index = dict((c, i) for i, c in enumerate(char_vocab)) # 문자에 고유한 정수 인덱스 부여
print(char_to_index)
index_to_char = {}
for key, value in char_to_index.items():
  index_to_char[value] = key
print(index_to_char)

{'!': 0, 'a': 1, 'e': 2, 'l': 3, 'p': 4}
{0: '!', 1: 'a', 2: 'e', 3: 'l', 4: 'p'}


In [57]:
# 입력 데이터와 레이블 데이터의 각 문자들을 정수로 맵핑
x_data = [char_to_index[c] for c in input_str]
y_data = [char_to_index[c] for c in label_str]
print(x_data)
print(y_data)

[1, 4, 4, 3, 2]
[4, 4, 3, 2, 0]


In [58]:
# nn.RNN()은 기본적으로 3차원 텐서를 입력 받는다.
# 배치 차원 추가
# 텐서 연산인 unsqueeze(0)을 통해 해결할 수도 있음.
x_data = [x_data]
y_data = [y_data]
print(x_data)
print(y_data)

[[1, 4, 4, 3, 2]]
[[4, 4, 3, 2, 0]]


In [59]:
# 입력 시퀀스의 각 문자들을 원-핫 벡터로 바꿔준다
x_one_hot = [np.eye(vocab_size)[x] for x in x_data]
print(x_one_hot)

[array([[0., 1., 0., 0., 0.],
       [0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 1.],
       [0., 0., 0., 1., 0.],
       [0., 0., 1., 0., 0.]])]


In [63]:
X = torch.FloatTensor(x_one_hot)
Y = torch.LongTensor(y_data)

print('훈련 데이터의 크기 : {}'.format(X.shape))
print('레이블의 크기 : {}'.format(Y.shape))

훈련 데이터의 크기 : torch.Size([1, 5, 5])
레이블의 크기 : torch.Size([1, 5])


In [61]:
# 2) 모델 구현하기
class Net(torch.nn.Module):
  def __init__(self, input_size, hidden_size, output_size):
    super(Net, self).__init__()
    self.rnn = torch.nn.RNN(input_size, hidden_size, batch_first=True)
    self.fc = torch.nn.Linear(hidden_size, output_size, bias=True)

  def forward(self, x):
    x, _status = self.rnn(x)
    x = self.fc(x)
    return x

net = Net(input_size, hidden_size, output_size)
outputs = net(X)
print(outputs.shape)

torch.Size([1, 5, 5])


In [64]:
criterion = torch.nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters(), learning_rate)

for i in range(100):
  optimizer.zero_grad()
  outputs = net(X)
  loss = criterion(outputs.view(-1, input_size), Y.view(-1)) # Batch 차원 제거를 위해
  loss.backward()
  optimizer.step()

  # 아래 줄은 모델이 실제 어떻게 예측했는지를 확인하기 위한 코드
  result = outputs.data.numpy().argmax(axis=2) # 최종 예측값인 각 time-step 별 5차원 벡터에 대해서 가장 높은값의 인덱스 선택
  result_str = ''.join([index_to_char[c] for c in np.squeeze(result)])
  print(i, 'loss: ', loss.item(), 'prediction: ', result, 'true Y:', y_data, 'prediction str: ', result_str)

0 loss:  1.5643374919891357 prediction:  [[4 4 4 2 4]] true Y: [[4, 4, 3, 2, 0]] prediction str:  pppep
1 loss:  1.2644165754318237 prediction:  [[4 4 0 2 0]] true Y: [[4, 4, 3, 2, 0]] prediction str:  pp!e!
2 loss:  1.0176327228546143 prediction:  [[4 4 4 2 0]] true Y: [[4, 4, 3, 2, 0]] prediction str:  pppe!
3 loss:  0.7850898504257202 prediction:  [[4 4 3 2 0]] true Y: [[4, 4, 3, 2, 0]] prediction str:  pple!
4 loss:  0.5807846784591675 prediction:  [[4 4 4 2 0]] true Y: [[4, 4, 3, 2, 0]] prediction str:  pppe!
5 loss:  0.4233388900756836 prediction:  [[4 4 3 2 0]] true Y: [[4, 4, 3, 2, 0]] prediction str:  pple!
6 loss:  0.30705526471138 prediction:  [[4 4 3 2 0]] true Y: [[4, 4, 3, 2, 0]] prediction str:  pple!
7 loss:  0.22082757949829102 prediction:  [[4 4 3 2 0]] true Y: [[4, 4, 3, 2, 0]] prediction str:  pple!
8 loss:  0.15697558224201202 prediction:  [[4 4 3 2 0]] true Y: [[4, 4, 3, 2, 0]] prediction str:  pple!
9 loss:  0.11162887513637543 prediction:  [[4 4 3 2 0]] true Y: 

In [66]:
# 2. 더 많은 데이터로 학습한 문자 단위 RNN
import torch
import torch.nn as nn
import torch.optim as optim

# 1) 훈련 데이터 전처리하기
sentence = ("if you want to build a ship, don't drum up people together to "
            "collect wood and don't assign them tasks and work, but rather "
            "teach them to long for the endless immensity of the sea.")
char_set = list(set(sentence)) # 중복을 제거한 문자 집합 생성
char_dic = {c: i for i, c in enumerate(char_set)} # 각 문자에 정수 인코딩
print(char_dic)
dic_size = len(char_dic)
print('문자 집합의 크기 : {}'.format(dic_size))

{'y': 0, 'm': 1, 't': 2, 'b': 3, 'l': 4, 'w': 5, 'f': 6, "'": 7, 'a': 8, 'p': 9, '.': 10, 'i': 11, 'k': 12, 'h': 13, 'n': 14, 'r': 15, 's': 16, 'c': 17, 'd': 18, 'u': 19, 'g': 20, 'e': 21, ' ': 22, 'o': 23, ',': 24}
문자 집합의 크기 : 25


In [67]:
# 하이퍼파라미터 설정
hidden_size = dic_size
sequence_length = 10 # 임의 숫자 지정
learning_rate = 0.1

# 데이터 구성
x_data = []
y_data = []

for i in range(0, len(sentence)- sequence_length):
  x_str = sentence[i:i + sequence_length]
  y_str = sentence[i + 1: i + sequence_length + 1]
  print(i, x_str, '->', y_str)

  x_data.append([char_dic[c] for c in x_str])
  y_data.append([char_dic[c] for c in y_str])

0 if you wan -> f you want
1 f you want ->  you want 
2  you want  -> you want t
3 you want t -> ou want to
4 ou want to -> u want to 
5 u want to  ->  want to b
6  want to b -> want to bu
7 want to bu -> ant to bui
8 ant to bui -> nt to buil
9 nt to buil -> t to build
10 t to build ->  to build 
11  to build  -> to build a
12 to build a -> o build a 
13 o build a  ->  build a s
14  build a s -> build a sh
15 build a sh -> uild a shi
16 uild a shi -> ild a ship
17 ild a ship -> ld a ship,
18 ld a ship, -> d a ship, 
19 d a ship,  ->  a ship, d
20  a ship, d -> a ship, do
21 a ship, do ->  ship, don
22  ship, don -> ship, don'
23 ship, don' -> hip, don't
24 hip, don't -> ip, don't 
25 ip, don't  -> p, don't d
26 p, don't d -> , don't dr
27 , don't dr ->  don't dru
28  don't dru -> don't drum
29 don't drum -> on't drum 
30 on't drum  -> n't drum u
31 n't drum u -> 't drum up
32 't drum up -> t drum up 
33 t drum up  ->  drum up p
34  drum up p -> drum up pe
35 drum up pe -> rum up peo
36

In [50]:
print(x_data[0])
print(y_data[0])

[11, 6, 22, 0, 23, 19, 22, 5, 8, 14]
[6, 22, 0, 23, 19, 22, 5, 8, 14, 2]


In [68]:
x_one_hot = [np.eye(dic_size)[x] for x in x_data]
X = torch.FloatTensor(x_one_hot)
Y = torch.LongTensor(y_data)

In [71]:
print('훈련 데이터의 크기 : {}'.format(X.shape))
print('레이블의 크기 : {}'.format(Y.shape))
print(X[0])
print(Y[0])

훈련 데이터의 크기 : torch.Size([170, 10, 25])
레이블의 크기 : torch.Size([170, 10])
tensor([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 1., 0., 0.],
        [1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 1., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 1., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 1., 0., 0.],
        [0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0.

In [72]:
# 모델 구현하기
class Net(torch.nn.Module):
  def __init__(self, input_dim, hidden_dim, layers):
    super(Net, self).__init__()
    self.rnn = torch.nn.RNN(input_dim, hidden_dim, num_layers=layers, batch_first=True)
    self.fc = torch.nn.Linear(hidden_dim, hidden_dim, bias=True)

  def forward(self, x):
    x, _status = self.rnn(x)
    x = self.fc(x)
    return x

net = Net(dic_size, hidden_size, 2) # 층을 2개 쌓음

# 비용 함수
criterion = torch.nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters(), learning_rate)

outputs = net(X)
print(outputs.shape)

torch.Size([170, 10, 25])


In [74]:
for i in range(100):
  optimizer.zero_grad()
  outputs = net(X)
  loss = criterion(outputs.view(-1, dic_size), Y.view(-1))
  loss.backward()
  optimizer.step()

  # results의 텐서 크기는 (170, 10)
  results = outputs.argmax(dim=2)
  predict_str = ""
  for j, result in enumerate(results):
    if j == 0: # 처음에는 예측 결과를 전부 가져오지만
      predict_str += ''.join([char_set[t] for t in result])
    else: # 그 다음에는 마지막 글자만 반복 추가
      predict_str += char_set[result[-1]]

  print(predict_str)

f you want to build a ship, don't arum up people together to collect wood and don't assign them tasks and work, but rather teach them to long for the endless immensity of the sea.
m you want to build a ship, don't arum up people together to collect wood and don't assign them tasks and work, but rather teach them to long for the endless immensity of the sea.
t you want to build a ship, don't arum up people together to collect wood and don't assign them tasks and work, but rather teach them to long for the endless immensity of the sea.
m you want to build a ship, don't arum up people together to collect wood and don't assign them tasks and work, but rather teach them to long for the endless immensity of the sea.
m you want to build a ship, don't arum up people together to collect wood and don't assign them tasks and work, but rather teach them to long for the endless immensity of the sea.
t you want to build a ship, don't arum up people together to collect wood and don't assign them task